# Module 1 - Dans ce notebook jupyter, du rééchantillonnage  et du masquage sont effectués afin de préparer les données pour des analyses ultérieures. 
* Etape 1a - Importer les modules/librairies
* Etape 1b - Rééchantillonner les données 
* Etape 1c - Filtrer les zone non-cultivées 

**=====================================================================================================================**

![title](img/Fig1_1.png)

**=====================================================================================================================**

## Etape 1a - Importer les modules/librairies

In [1]:
import os                             # module d'interaction avec le système d'exploitation
import glob                           # utilisé pour retrouver des fichiers/noms de chemin correspondant à un shéma donné
import matplotlib.pyplot as plt       # Librairie de tracage de graphiques 2D dans python 
import numpy as np                    # Signifie 'Numerical Python' c'est une librairie de python utilisée pour les calculs scientifiques avec des matrices
from osgeo import ogr, gdal
import subprocess

os.chdir(os.path.join(os.path.split(os.getcwd())[0], "Modules"))
from GIS_functions import GIS_function as gis

# Etape 1b - Rééchantilloner les données raster

## i) Rééchantilloner l'ET de référence

## * Importer les données d'entrée 

In [2]:
dir_proj = os.path.split(os.getcwd())[0] 
dir_data = "Data"
    
source_file   = os.path.join(dir_proj, dir_data,   "WAPOR.v2_mm-dekad-1_L2_AETI_D", "L2_AETI_0901.tif") # Lire l'info gdal du canevas d'un fichier raster  
target_folder = os.path.join(dir_proj, dir_data,   "WAPOR.v2_mm-dekad-1_L1_RET_D")                      # donnée à rééchantillonner 
target_fhs    = glob.glob(target_folder + '\*.tif')

source_file, target_fhs

## La taille et la forme des fichiers raster

In [3]:
## La taille et la forme des fichiers raster
template   = gis.OpenAsArray(source_file, nan_values=True) 
original   = gis.OpenAsArray(target_fhs[0], nan_values=True)

print ('The size & shape of the template raster      =', template.size,  '&', template.shape)
print ('The size & shape of the data to be resampled =', original.size,  '&', original.shape)

## ** Créér ou connecter le dossier de sortie avec le repertoire

In [4]:
dir_proj = os.path.split(os.getcwd())[0] 
dir_data = "Data"

output_folder = os.path.join(dir_proj, dir_data, "1_L1_RET_D_resampled") 

## En créer un, si le dossier n'existe pas
if not os.path.exists(output_folder):
    os.makedirs(output_folder) 
    
output_folder

## *** Rééchantilloner les données raster

In [5]:
Resample = gis.MatchProjResNDV (source_file, target_fhs, output_folder, resample = 'near', dtype = 'float32')

In [5]:
## La taille et la forme des fichiers raster rééchantillonnés
Resampled   = os.path.join(dir_proj, dir_data,   "1_L1_RET_D_resampled", "L1_RET_0901.tif") 
resampled   = gis.OpenAsArray(Resampled , nan_values=True)

print ('The size & shape of the resampled data =', resampled.size,  '&', resampled.shape)

## ii) Rééchantillonner les couches de Précipitation

## * Importer les données d'entrée

In [ ]:
dir_proj = os.path.split(os.getcwd())[0] 
dir_data = "Data"
    
source_file   = os.path.join(dir_proj, dir_data,   "WAPOR.v2_mm-dekad-1_L2_AETI_D", "L2_AETI_0901.tif") # Lire l'info gdal du canevas d'un fichier raster  
target_folder = os.path.join(dir_proj, dir_data,   "WAPOR.v2_mm-dekad-1_L1_PCP_D")                      # données à rééchantillonner
target_fhs    = glob.glob(target_folder + '\*.tif')

source_file, target_fhs

## La taille et la forme des fichiers raster

In [ ]:
## La taille et la forme des données raster
template   = gis.OpenAsArray(source_file, nan_values=True) 
original   = gis.OpenAsArray(target_fhs[0], nan_values=True)

print ('The size & shape of the template raster      =', template.size,  '&', template.shape)
print ('The size & shape of the data to be resampled =', original.size,  '&', original.shape)

## ** Créér ou connecter avec le dossier des données de sortie

In [ ]:
dir_proj = os.path.split(os.getcwd())[0] 
dir_data = "Data"

output_folder = os.path.join(dir_proj, dir_data, "1_L1_PCP_D_resampled") 

## En créer un, si le dossier n'existe pas
if not os.path.exists(output_folder):
    os.makedirs(output_folder) 
    
output_folder

## *** Rééchantilloner les données raster

In [ ]:
Resample = gis.MatchProjResNDV (source_file, target_fhs, output_folder, resample = 'near', dtype = 'float32')

In [ ]:
## La taille et la forme des fichiers raster rééchantillonnés
Resampled   = os.path.join(dir_proj, dir_data,   "1_L1_RET_D_resampled", "L1_RET_0901.tif") 
resampled   = gis.OpenAsArray(Resampled , nan_values=True)

print ('The size & shape of the resampled data =', resampled.size,  '&', resampled.shape)



# Etape 1c - Filtrer les zones non cultivées en utilisant la carte d'occupation des terres et les limites du projet

### Carte d'occupation des terres
Dans la couche d'occupation des terres de WaPOR, la valeur de pixel **42** représents les terres agricoles irriguées (Voir https://wapor.apps.fao.org/catalog/2/L2_LCC_A). 

![title](img/Fig1_2.png)

### Limites du projet  --- convertir un shapefile en raster

In [6]:
# Lire les fichier shapefile et raster de référence
dir_proj = os.path.split(os.getcwd())[0] 
dir_data = "Data"

InputVector = r"..\Data\1Boundary\Shapefile\Xinavane_1.shp"             # Le shapefile du projet
RefImage    = r"..\Data\WAPOR.v2_mm-dekad-1_L2_AETI_D\L2_AETI_0901.tif" # Raster de rérérence pour fixer la taille des pixels 
OutputImage = r'..\Data\1Boundary\mask_irrigation_types.tif'            # nommer le fichier raster de sorite 'mask_irrigation_types' et l'enregistrer dans DAta/1Boundary..
burnVal     = 1                                                         #Valeur pour les pixels de l'image de sortie

In [7]:
# Un code pour transformer un shapefile en raster en respectant la même projection et résolution de pixel qu'une image de référence.
gdalformat = 'GTiff'
datatype = gdal.GDT_Byte


##########################################################
# Obtenir l'info de la projection à partir de l'image de référence
Image = gdal.Open(RefImage, gdal.GA_ReadOnly)

# Ouvrir le Shapefile
Shapefile = ogr.Open(InputVector)
Shapefile_layer = Shapefile.GetLayer()

# Transformer en raster
print("Rasterising shapefile...")
Output = gdal.GetDriverByName(gdalformat).Create(OutputImage, Image.RasterXSize, Image.RasterYSize, 1, datatype, options=['COMPRESS=LZW'])
Output.SetProjection(Image.GetProjectionRef())
Output.SetGeoTransform(Image.GetGeoTransform()) 

# Ecrire les donnée à la bande 1
Band = Output.GetRasterBand(1)
Band.SetNoDataValue(0)
gdal.RasterizeLayer(Output, [1], Shapefile_layer, burn_values=[burnVal])

# Fermer les ensembles de données
Band = None
Output = None
Image = None
Shapefile = None

# Construire les apperçus des images
subprocess.call("gdaladdo --config COMPRESS_OVERVIEW DEFLATE "+OutputImage+" 2 4 8 16 32 64", shell=True)
print("Done.")

## i) Importer les données d'entrée 

In [10]:
dir_proj = os.path.split(os.getcwd())[0]  
dir_data = "Data"

# transpiration, évapotranspiration & interception and évapotranspiration de référence
input_folderT    = os.path.join(dir_proj, dir_data, "WAPOR.v2_mm-dekad-1_L2_T_D") 
input_fhsT       = glob.glob(input_folderT + '\*.tif')  # glob.glob retourne la liste des fichiers avec leur chemins au complet

input_folderAETI = os.path.join(dir_proj, dir_data, "WAPOR.v2_mm-dekad-1_L2_AETI_D") 
input_fhsAETI    = glob.glob(input_folderAETI + '\*.tif')   

input_folderRET  = os.path.join(dir_proj, dir_data, "1_L1_RET_D_resampled") 
input_fhsRET     = glob.glob(input_folderRET + '\*.tif')

input_folderNPP  = os.path.join(dir_proj, dir_data, "WAPOR.v2_mm-dekad-1_L2_NPP_D") 
input_fhsNPP     = glob.glob(input_folderNPP + '\*.tif')

# Masques: Couches d'occupation des terres et forme de ma zone du projet
input_LCCfolder  = os.path.join(dir_proj, dir_data, "WAPOR.v2_LCC_L2_LCC_A") 
LCC_tifs         = glob.glob(input_LCCfolder + '\*.tif')

ProArea = gis.OpenAsArray(r'..\Data\1Boundary\mask_irrigation_types.tif', nan_values=True)  # les limites (aire) de la zone du projet(tif)

## ii) Dossier de sortie: En créer un ou connecter l'existant

In [8]:
dir_proj = os.path.split(os.getcwd())[0] 
dir_data = "Data"

output_folderT    = os.path.join(dir_proj, dir_data, "1_L2_T_filtered")
output_folderAETI = os.path.join(dir_proj, dir_data, "1_L2_AETI_filtered")
output_folderRET  = os.path.join(dir_proj, dir_data, "1_L1_RET_filtered")
output_folderNPP  = os.path.join(dir_proj, dir_data, "1_L2_NPP_filtered")
 

## En créer un, si le dossier n'existe pas
if not os.path.exists(output_folderT):
    os.makedirs(output_folderT) 
if not os.path.exists(output_folderAETI):
    os.makedirs(output_folderAETI) 
if not os.path.exists(output_folderRET):
    os.makedirs(output_folderRET) 
if not os.path.exists(output_folderNPP):
    os.makedirs(output_folderNPP) 
    
output_folderAETI, output_folderRET

#### Tracer la zone du projet

In [9]:
### Traver la zone du projet 
Projectboundary = ProArea

# Tracer
plt.figure(figsize = (12,6))
plt.imshow(Projectboundary)                          
plt.title('Project area')
plt.show();

#### Tracer la couche raster (avant de retirer aux zones non cultivées)

In [10]:
input_fhsAETI[0]

In [11]:
# Tracer la couche de l'ET de référence 
AETI_tif = gis.OpenAsArray(input_fhsAETI[0],nan_values=True)

plt.figure(figsize = (12,8))
plt.imshow(AETI_tif, cmap='RdYlGn')
plt.colorbar(shrink=0.75, label=' [mm/dekadal]')
plt.title('Downloaded AETI layer')
plt.show()

#### Tracer la couche raster sur les terres cultivées irriguées (masquage des terres cultivées non irriguées)

In [12]:
LCC_tifs[0]

In [13]:
# Masquer les terres cultivées non irriguées
LCC       = gis.OpenAsArray(LCC_tifs[0],nan_values=True)  # carte d'occupation des terres
AETI_crop = np.where((LCC==42), AETI_tif,np.nan)          # Montrer l'ETRI des terres cultivées irriguées, lesquelles on la valeur de pixel 42

# Tracer l'ETRI des terres cultivées irriguées a
plt.figure(figsize = (12,8))
plt.imshow(AETI_crop, cmap='RdYlGn')    
plt.colorbar(shrink=0.75, label=' [mm/dekadal]')                          
plt.title('AETI of the irrigated cropped land')
plt.show()

#### Tracer la couche raster des terres cultivées irriguées dans les limites du périmètre

In [14]:
# Masquer les terres cultivées non irriguées et les zone hors des limites du périmètre
AETI_boundary = np.where((LCC==42)& (ProArea==1),AETI_tif,np.nan) # Masquage des terres cultivées non irriguées et les zone hors des limites du la zone du projet

# Plot
plt.figure(figsize = (12,8))
plt.imshow(AETI_boundary, cmap='RdYlGn')   
plt.colorbar(shrink=0.75, label=' [mm/dekadal]')                          
plt.title('AETI of the irrigated project')
plt.show()

## iii) Filtrer les terres non cultivées en utilisant la carte d'occupation des terres et les limites du projet

* LCC     = par exemple # 42 est la classe d'occupation des terres sous gestion d'eau
* ProArea = 1 la valeur du raster de la zone du projet au format tiff

### Filtrer les couches de transpiration

In [15]:
# collecter les Geoinfo tels que la projection, les axes x et y
in_fh = input_fhsT[0]     
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

for Tfh in input_fhsT: 
    for LCCfh in LCC_tifs:
        T   = gis.OpenAsArray(Tfh,  nan_values=True)
        LCC = gis.OpenAsArray(LCCfh, nan_values=True)
        basename1  = os.path.basename(Tfh).split('_')[2].split('.')[0]    #L2_T_0901.tif, 0901.tif, 0901
        basename1  = int(str(basename1)[:2])                              # 09

        basename2  = os.path.basename(LCCfh).split('_')[2].split('.')[0]   #L2_LCC_09.tif, 09.tif, 09
        
        if basename1 == int(basename2):     #Compare l'année de la couche d'occupation des terres et la couche raster (T)
            
            # Masquage des terres cultivées non irriguées et les zone hors des limites du la zone du projet
            T   = np.where((LCC==42)& (ProArea==1),T,np.nan) 
            
            # mettre à jour le nom du fichier, et l'enrégistrer dans le dossier de sortie
            basename  = os.path.basename(Tfh)
            output_fn = os.path.join(output_folderT, basename)
            gis.CreateGeoTiff(output_fn, T, driver, NDV, xsize, ysize, GeoT, Projection) 

### Filtrer les couches d'évapotranspiration

In [16]:
# collecter les Geoinfo tels que la projection, les axes x et y
in_fh = input_fhsAETI[0]   
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

for AETIfh in input_fhsAETI:
    for LCCfh in LCC_tifs:
        AETI = gis.OpenAsArray(AETIfh,  nan_values=True)
        LCC = gis.OpenAsArray(LCCfh, nan_values=True)
        basename1  = os.path.basename(AETIfh).split('_')[2].split('.')[0]   
        basename1  = int(str(basename1)[:2])                             

        basename2  = os.path.basename(LCCfh).split('_')[2].split('.')[0]   
        
        if basename1 == int(basename2):     #Compare l'année de la couche d'occupation des terres et la couche raster (ETRI)
            
            # Masquage des terres cultivées non irriguées et les zone hors des limites du la zone du projet
            AETI = np.where((LCC==42)& (ProArea==1),AETI,np.nan)     
    
            # mettre à jour le nom du fichier, et l'enrégistrer dans le dossier de sortie
            basename  = os.path.basename(AETIfh)
            output_fn = os.path.join(output_folderAETI, basename)
            gis.CreateGeoTiff(output_fn, AETI, driver, NDV, xsize, ysize, GeoT, Projection) 

### Filtrer les couches de l'évapotranspiration de référence

In [17]:
# ollecter les Geoinfo tels que la projection, les axes x et y
in_fh = input_fhsRET[0]   
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster
for RETfh in input_fhsRET:
    for LCCfh in LCC_tifs:
        RET = gis.OpenAsArray(RETfh,  nan_values=True)
        LCC = gis.OpenAsArray(LCCfh, nan_values=True)
        basename1  = os.path.basename(RETfh).split('_')[2].split('.')[0]   
        basename1  = int(str(basename1)[:2])                              

        basename2  = os.path.basename(LCCfh).split('_')[2].split('.')[0]  
        
        if basename1 == int(basename2):     #compare l'année de la couche d'occupation des terres et la couche raster (évapotranspiration de référence)
            
            # Masquage des terres cultivées non irriguées et les zone hors des limites du la zone du projet
            RET = np.where((LCC==42)& (ProArea==1),RET,np.nan)     
        
            # mettre à jour le nom du fichier, et l'enrégistrer dans le dossier de sortie
            basename  = os.path.basename(RETfh)
            output_fn = os.path.join(output_folderRET, basename)
            gis.CreateGeoTiff(output_fn, RET, driver, NDV, xsize, ysize, GeoT, Projection) 

### Filtrer les couche Production Primaire Nette (NPP)

In [18]:
# ollecter les Geoinfo tels que la projection, les axes x et y
in_fh = input_fhsNPP[0]   
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster
for NPPfh in input_fhsNPP:
    for LCCfh in LCC_tifs:
        NPP = gis.OpenAsArray(NPPfh,  nan_values=True)
        LCC = gis.OpenAsArray(LCCfh, nan_values=True)
        basename1  = os.path.basename(NPPfh).split('_')[2].split('.')[0]    
        basename1  = int(str(basename1)[:2])                             

        basename2  = os.path.basename(LCCfh).split('_')[2].split('.')[0]   
        
        if basename1 == int(basename2):     
            
            # Masquage des terres cultivées non irriguées et les zone hors des limites du la zone du projet
            NPP = np.where((LCC==42)& (ProArea==1),NPP,np.nan)      
            
            # mettre à jour le nom du fichier, et l'enrégistrer dans le dossier de sortie
            basename  = os.path.basename(NPPfh)
            output_fn = os.path.join(output_folderNPP, basename)
            gis.CreateGeoTiff(output_fn, NPP, driver, NDV, xsize, ysize, GeoT, Projection) 